In [1]:
# CHANGE THIS !!!

from datasets import load_dataset
dataset = load_dataset('csv', data_files='pairs.csv', column_names=['bo', 'phon'])

/home/j/Documents/MLotsawa/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = dataset['train'].train_test_split(.15)

In [3]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [4]:
dataset['train'][1]

{'bo': 'ད་རེས་རང་གིས་རང་ཉིད་མ་བསླུས་ན།།',
 'phon': 'daré rang gi rangnyi ma lü na'}

In [5]:
source_lang = 'bo'
target_lang = 'phon'
prefix = "Transliterate: "

def preprocess_function(examples):

    inputs = [prefix + example for example in examples[source_lang]]
    targets = [example for example in examples[target_lang]]
    
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)

    return model_inputs


In [6]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 14790/14790 [00:00<00:00, 50191.68 examples/s]


In [7]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback, Adafactor

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="auto")

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=True, 
    relative_step=False, 
    warmup_init=False, 
    lr=3e-4
)

In [8]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer = accelerator.prepare(model, optimizer)

In [9]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"../../models/transliteration/",
    auto_find_batch_size=True,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    num_train_epochs=100
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    optimizers=(optimizer, None),
    data_collator=data_collator
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


  0%|          | 503/1047600 [00:35<20:42:05, 14.05it/s]

{'loss': 3.37, 'grad_norm': 1.9563603401184082, 'learning_rate': 0.00029985681557846504, 'epoch': 0.05}


  0%|          | 1003/1047600 [01:08<19:37:36, 14.81it/s]

{'loss': 2.808, 'grad_norm': 1.1998969316482544, 'learning_rate': 0.0002997136311569301, 'epoch': 0.1}


...

{'loss': 0.9995, 'grad_norm': 2.6112301349639893, 'learning_rate': 0.0001994845360824742, 'epoch': 33.51}


 34%|███▎      | 351503/1047600 [6:49:06<12:37:19, 15.32it/s]

{'loss': 1.0334, 'grad_norm': 2.362074851989746, 'learning_rate': 0.0001993413516609393, 'epoch': 33.55}


 34%|███▎      | 351847/1047600 [6:49:30<15:07:35, 12.78it/s]

KeyboardInterrupt: 